In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from glob import glob

In [ ]:
# Image size
IMAGE_SIZE = [224, 224]

train_path = 'data/train'
valid_path = 'data/validation'

densenet = DenseNet121(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# Freeze all layers
for layer in densenet.layers:
    layer.trainable = False

folders = glob('data/train/*')

In [ ]:
# Add layer
x = Flatten()(densenet.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=densenet.input, outputs=prediction)


In [ ]:
# # Memastikan tidak ada karakter '/' di nama layer
for layer in model.layers:
  if '/' in layer.name:
    layer._name = layer.name.replace('/', '-')

In [ ]:
# Setup learning rate
initial_learning_rate = 0.0001
optimizer = Adam(learning_rate=initial_learning_rate)


model.compile(
  loss='categorical_crossentropy',
  optimizer=optimizer,
  metrics=['accuracy']
)


train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(224, 224),
                                            batch_size=16,
                                            class_mode='categorical')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Set up callbacks
checkpoint = ModelCheckpoint('result/best_model_densenet.keras',
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              min_lr=1e-6,
                              verbose=1)

callbacks = [checkpoint, reduce_lr]

In [ ]:
# Train the model
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=15,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=callbacks
)

## Predict

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np


# Fungsi untuk memproses gambar
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisasi
    return img_array

# Path ke img
img_path = 'result/telur-dadar.jpg'

# Preprocessing gambar
img_array = preprocess_image(img_path)

# Prediksi
predictions = model.predict(img_array)

# Menampilkan hasil prediksi
predicted_class = np.argmax(predictions, axis=1)[0]

# Mapping indeks kelas ke label kelas
class_indices = training_set.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Menampilkan label kelas yang diprediksi
predicted_label = index_to_class[predicted_class]
print(f"Predicted class: {predicted_label}")

In [ ]:
# Menampilkan semua prediksi dengan probabilitasnya
print("All class predictions with probabilities:")
for idx, prob in enumerate(predictions[0]):
    label = index_to_class[idx]
    print(f"Class index: {idx}, label: {label}, probability: {prob:.6f}")

## Saving

In [ ]:
model.save('result/best_model_densenet_27.keras')

In [ ]:
model.summary()